# Classifying the Kaggle Dataset for Distracted Driver Competition

# This program is used to recognize the driver's status (one of the 10 statuses) based on the image using pre-trained VGG16  deep convolutional neural network (CNN).

The code in the Program will do the fine tuning of pre trained VGG16 and did some changes like:-

## the lower model: layer 0-layer24 of the original VGG16 net  (frozen the first 4 blocks, train the weights of the 5-th block with our dataset)

## the upper model: newly added two layer dense net (train the weights using our dataset)



In [3]:
#importing the neccesary libraries

import os
# for reading the binary data of Neural network weights for faster processing
import h5py
#Keras is a high-level neural networks API, written in Python for developing basic black box modules of network ( like activation or maxpool layer )
#instantly for fast prototyping
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
# Sequential function consist of modules for making neurons
from keras.models import Sequential
## for  prototyping architecture and implementation functions
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense

# writing the directory path of files which are to be read , download the weights from https://drive.google.com/file/d/0Bz7KyqmuGsilT0J5dmRCM0ROVHc/view
weights_path = 'vgg16_weights.h5'
img_width , img_height = 150,150

train_data_dir = 'imgs/train'

validation_data_dir = 'imgs/validation'
# number of images present per feature to train

nb_train_samples = 20924

# for validation
nb_validation_samples = 1500
# for number of total epochs ( one epoch is equal to forward and backward pass done for error calculation)
nb_epoch = 10


Using TensorFlow backend.


# Now Implementing VGG Model

![MicroArchitecture VGG16](vgg16.png)

## It Consist of Multiple Layers of Convolutional and activation Layer  and gives top results , 

## Implemented from "   Very Deep Convolutional Networks for Large-Scale Image Recognition :- K. Simonyan, A. Zisserman  arXiv:1409.1556" 



In [15]:
# implementing the model using keras sequential function , also to avoid the error :- "negative dimension size ...
# we have used dim_ordering according to the backend framework you are using 



model = Sequential()
model.add(ZeroPadding2D((1, 1), input_shape=(3, img_width, img_height)))

model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2),data_format="channels_first"))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2),data_format="channels_first"))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2),data_format="channels_first"))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2),data_format="channels_first"))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2),data_format="channels_first"))






ValueError: It seems that you are using the Keras 2 and you are passing both `kernel_size` and `strides` as integer positional arguments. For safety reasons, this is disallowed. Pass `strides` as a keyword argument instead.

In [6]:
# load the weights of the VGG16 networks (trained on ImageNet, won the ILSVRC competition in 2014 and got near human accuracy)



assert os.path.exists(weights_path)
# load the weights for each layer from the binary file  of pre-trained vgg16
f = h5py.File(weights_path)

#now for looping for each layer of pretrained vgg (except of fully connected layer )
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers):
       
        break
    # now for each pass , we will be setting the weights , we will generate two 1d vectors consisting of layer number
    #and corresponding parameters 
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]

    model.layers[k].set_weights(weights)
f.close()
print('VGG16 Model parameters loaded')

    

ValueError: Layer weight shape (3, 3, 150, 64) not compatible with provided weight shape (64, 3, 3, 3)

In [ ]:
# Please Solve Previous problem  by putting a pull request

#now making a classifier

top_model = Sequential()
# CONVERTING  the output of VGG16 model to 2D Numpy matrix (n*D)
top_model.add(Flatten(input_shape=model.output_shape[1:]))
# hidden layer of 256 neurons
top_model.add(Dense(256, activation='relu'))
# add dropout for the dense layer
top_model.add(Dropout(0.5))
# Creating the output of the layer having 10 classes
top_model.add(Dense(10, activation='softmax'))




# connect the two models onto the VGG16 net
model.add(top_model)


# as the initial 25 layers are not required , hence they will not be trained

for layer in model.layers[:25]:
    layer.trainable=False

    
#Doing Image Preprocessing  for traning and test data:-


train_datagen = ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Now traning the  images and generating the data
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_height, img_width), 
batch_size=32, class_mode='categorical')

#fit the model from both train_gen and test_gen and save the weights


model.fit_generator(train_generator, samples_per_epoch=nb_train_samples, nb_epoch=nb_epoch, 
validation_data=validation_generator, nb_val_samples=nb_validation_samples)

model.save_weights('VGG16_MLP_model.h5')